[Home](../index.ipynb) / Web-Server: Simple webserver with files
***

# Simple webserver with files

In [ ]:
#%serialconnect --port=COM3 # Windows
%serialconnect # Linux or Windows with one COM port 

#######################################
# Constants

# ID_LED = 13 # Esp8266: Croduino 
ID_LED = 25 # Esp32 HeltTec
# ID_LED = 14 # Esp32 NodeMCU, external LED

#======================================
SSID        = "ESP minimal http-server"
PASSWORD    = '123123123123' # default is 'micropythoN', attension: at least 8 chars, otherwise error: 'OSError: can't set AP config'
MAX_CLIENTS = 5 # Good maximal ammount of posiible clients


POLL_TIME_MILLIS = 0

#######################################
# Init

from machine import PWM, Pin

led = PWM( Pin(25, Pin.OUT), freq=1000, duty=0)  # 25 HelTec, 13 Croduino


#======================================
# Typical code for setting up the controller as access point.
# At the end we get a server-socket listening for clients to connect:

import network
import socket # usocket
import select # uselect


wlanAccessPoint = network.WLAN( network.AP_IF ) # create access-point interface

wlanAccessPoint.config( essid = SSID, password = PASSWORD ) # ESP-8266 and ESP-32
# wlanAccessPoint.config( essid = SSID, password = PASSWORD, max_clients = MAX_CLIENTS ) # ESP32
    
wlanAccessPoint.active( True )  # activate the interface

print( wlanAccessPoint.ifconfig()[0] )

# A internet stream server-socket: this server-socket creates the clientSockets.
serverSocket = socket.socket( socket.AF_INET, socket.SOCK_STREAM )

# To prevent "OSError: [Errno 98] Address already in use",
# when a previous execution has left the socket in a TIME_WAIT state,
# and so can’t be immediately reused:

#   SOL_SOCKET:   Socket Options That Apply to the Socket Layer
#   SO_REUSEADDR: Indicates if the local socket address can be reused.
#                 This option is supported by sockets with an address family
#                 of AF_INET or AF_INET6 and a type of SOCK_STREAM or SOCK_DGRAM.
#   1:            A value of 0 (default) disables the option.
#                 A non-zero value sets the option indicating the local socket address can be reused.

serverSocket.setsockopt( socket.SOL_SOCKET, socket.SO_REUSEADDR, 1 )

try:
    serverSocket.bind( ( '', 80 ) ) # everyone can connect
    serverSocket.listen( MAX_CLIENTS )
except Exception as _e:
    print( _e )


# Register serverSocket for polling (POLLIN: only poll for input)
poll = select.poll()
poll.register( serverSocket, select.POLLIN )


#######################################
# Extract request-path from header:
# For example:
#   In browser:      "http://192.168.4.1/led/On"
#   ==> strRequest:  "GET /led/On"
#   This function removes the "GET " and returns in this example "/led/On".

def getPathFromRequest( strRequest ) :
    iRequestPathStart = strRequest.find( "GET " ) # Every http request starts with "GET "

    if iRequestPathStart >= 0 :                    # followed by the request-path delimited with a ' ' charakter.
        iRequestPathStart += 4

        iRequestPathEnd = strRequest.find( " ", iRequestPathStart )

        if iRequestPathEnd >= 0 :
            return strRequest[ iRequestPathStart : iRequestPathEnd ]

    return ""
    

def sendHttpHeader( strResponseType, strContentType, clientSocket ):
    clientSocket.send( "HTTP/1.1 " )
    clientSocket.send( strResponseType )
    clientSocket.send( "\nContent-Type: text/" )
    clientSocket.send( strContentType )
    clientSocket.send( "; charset=UTF-8\n" ) # we send html and text is encoded in UTF-8
    clientSocket.send( "Connection: close\n\n" ) # We are (suspending and) closing the connection after sending the data.

def sendFile( strFileName, clientSocket ):    
    with open( strFileName, 'r' ) as f:
        while True:
            chunk = f.read(1024) # Read the next 1KB chunk
            
            if not chunk:
                break
                
            clientSocket.send(chunk) # Send the next 1KB chunk
            
            
#######################################
# Loop
# Listen for connecting clients and
# handle requests
import time
import display
display = display.Display()

import re
reSpeed = re.compile( "/speed/(.*)" )


try:
    while True:
        aEvent = poll.poll( POLL_TIME_MILLIS ) # wait POLL_TIME_MILLIS, then give other tasks a chanche, to do there work.
        
        # check if we have stuff to handle
        
        if aEvent:
            # Only serverSocket is registered for polling, so only this one can be in the list (aEvent)
            
            try:
                # The return value is a pair (clientSocket, clientAddress)
                # where clientSocket is a new socket object where we can
                # receive data from and send data to:
                clientSocket, clientAddress = serverSocket.accept()

                strPath = getPathFromRequest( str( clientSocket.recv( 1024 ) ) ) # read maximal 1024 bytes
                
                print( "Path: ", strPath )
                
                match = reSpeed.match( strPath )
            
                if match :
                    iSpeed = int(match.group( 1 ))
                    led.duty( iSpeed*2 if iSpeed >= 0 else -iSpeed*2 )
                    
                    sendHttpHeader( "200 OK", "plain", clientSocket )
                    clientSocket.send( str(iSpeed) )
                   
                elif strPath == "/" :
                    sendHttpHeader( "200 OK", "html", clientSocket )
                    sendFile( "/index.html", clientSocket )
                             
                elif strPath == "/default.css" :
                    sendHttpHeader( "200 OK", "css", clientSocket )
                    sendFile( "/default.css", clientSocket )

                elif strPath == "/default.js" :
                    sendHttpHeader( "200 OK", "javascript", clientSocket )
                    sendFile( "/default.js", clientSocket )
                             
                else:
                    sendHttpHeader( "404 Not Found", "html", clientSocket )
                    sendFile( "/error404.html", clientSocket )

            except Exception as _e:
                print( _e )

            finally:
                try   : clientSocket.close()
                except: pass

        # Do something in the "background":
        display.fill_rect( 0, 20, 128, 40, 0 )
        display.text( "Time since start:", 0, 20 )
        display.text( "{:>.2f} s".format( time.ticks_ms()/1000 ), 0, 34 )
        display.show()
        
except KeyboardInterrupt:
    pass # User has typed "Ctrl C"

finally:
    # Clean up:
    try   : serverSocket.close()
    except: pass

    try   : wlanAccessPoint.active( False )  
    except: pass
    led.deinit()

    import gc
    gc.collect()

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.
192.168.4.1
Path:  /
Path:  /default.js
Path:  /default.css
.Path:  /
Path:  /default.css
Path:  /default.js
Path:  /speed/0
.Path:  /speed/3
Path:  /speed/-5
Path:  /speed/-84
Path:  /speed/-239
Path:  /speed/-500
.Path:  /speed/-493
Path:  /speed/-436
Path:  /speed/-267
Path:  /speed/50
Path:  /speed/500
.Path:  /speed/498
Path:  /speed/490
Path:  /speed/460
Path:  /speed/311
Path:  /speed/172
Path:  /speed/19
Path:  /speed/-119
Path:  /speed/-257
Path:  /speed/-344
Path:  /speed/-406
Path:  /speed/-446
Path:  /speed/-465
Path:  /speed/-468
Path:  /speed/-428
Path:  /speed/-340
Path:  /speed/-226
Path:  /speed/-64
Path:  /speed/98
Path:  /speed/247
Path:  /speed/305
Path:  /speed/303
Path:  /speed/215
Path:  /speed/97
Path:  /speed/-124
Path:  /speed/-253
Path:  /speed/-357
Path:  /speed/-398
Path:  /speed/-401
Path:  /speed/-314
Path:  /speed/-164
Path:  /speed/52
Path:  /speed/221
Path:  /speed/304
Path:  /speed/0
Path:  /spee